# CCP: Comparative Machine Learning Framework for Heart Disease Prediction

**Course:** COMP-240L Machine Learning Lab  
**Students:** Ali Hamza & Zarmeena Jawad  
**Student Numbers:** B23F0063AI106 & B23F0115AI125  
**Dataset:** Cleveland Heart Disease Dataset  
**Objective:** Develop and critically evaluate a predictive diagnostic system using multiple machine learning algorithms

## Project Overview

This project implements a comprehensive comparative machine learning framework to predict the presence or absence of heart disease using patient clinical data. The framework evaluates five different algorithms:

1. **Decision Tree (DT)**
2. **Random Forest (RF)**
3. **Support Vector Machine (SVM)**
4. **Artificial Neural Network (ANN)**
5. **K-Nearest Neighbors (KNN)**

The goal is to determine which model offers the best trade-off between accuracy, precision, and interpretability for medical diagnostic prediction.


## 1. Import Libraries and Setup

We'll import all necessary libraries for data processing, machine learning, and visualization.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                           precision_score, recall_score, f1_score, roc_auc_score,
                           roc_curve)
from imblearn.over_sampling import SMOTE
import joblib

warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✓ All libraries imported successfully!")


In [ ]:
# Column names for Cleveland Heart Disease dataset
column_names = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'
]

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"

try:
    df = pd.read_csv(url, names=column_names, na_values='?')
    print("✓ Dataset loaded successfully from UCI ML Repository")
except:
    # Try alternative source
    try:
        df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/heart.csv")
        print("✓ Dataset loaded from alternative source")
    except:
        # Load from local file if available
        df = pd.read_csv('../Data/heart_disease.csv', names=column_names, na_values='?')
        print("✓ Dataset loaded from local file")

# Convert target to binary (0 = no disease, 1-4 = disease)
df['target'] = (df['target'] > 0).astype(int)

print(f"\nDataset Shape: {df.shape}")
print(f"Features: {df.shape[1]-1}")
print(f"Instances: {df.shape[0]}")
df.head()


## 3. Exploratory Data Analysis (EDA)

Let's explore the dataset to understand its characteristics, distributions, and relationships between features.


In [ ]:
# Basic information
print("Dataset Info:")
print(df.info())
print("\n" + "="*70)
print("Statistical Summary:")
print(df.describe())
print("\n" + "="*70)
print("Missing Values:")
print(df.isnull().sum())
print("\n" + "="*70)
print("Target Distribution:")
print(df['target'].value_counts())
print(f"\nClass 0 (No Disease): {df['target'].value_counts()[0]} ({df['target'].value_counts()[0]/len(df)*100:.2f}%)")
print(f"Class 1 (Disease): {df['target'].value_counts()[1]} ({df['target'].value_counts()[1]/len(df)*100:.2f}%)")


In [ ]:
# Create comprehensive EDA visualizations
fig = plt.figure(figsize=(20, 14))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
fig.suptitle('Heart Disease Dataset - Exploratory Data Analysis', 
             fontsize=18, fontweight='bold', y=0.995)

# Target distribution
ax1 = fig.add_subplot(gs[0, 0])
target_counts = df['target'].value_counts()
colors = ['#ff9999', '#66b3ff']
ax1.bar(['No Disease', 'Disease'], target_counts.values, color=colors, alpha=0.7, edgecolor='black')
ax1.set_title('Target Variable Distribution', fontweight='bold')
ax1.set_ylabel('Count')
for i, v in enumerate(target_counts.values):
    ax1.text(i, v + 5, str(v), ha='center', fontweight='bold')

# Correlation heatmap
ax2 = fig.add_subplot(gs[0, 1:])
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
           ax=ax2, cbar_kws={'shrink': 0.8}, fmt='.2f', square=True)
ax2.set_title('Feature Correlation Matrix', fontweight='bold')

# Age distribution by target
ax3 = fig.add_subplot(gs[1, 0])
df.boxplot(column='age', by='target', ax=ax3)
ax3.set_title('Age Distribution by Disease Status', fontweight='bold')
ax3.set_xlabel('Disease Status')
ax3.set_ylabel('Age')
plt.setp(ax3.get_xticklabels(), rotation=0)

# Cholesterol distribution by target
ax4 = fig.add_subplot(gs[1, 1])
df.boxplot(column='chol', by='target', ax=ax4)
ax4.set_title('Cholesterol Distribution by Disease Status', fontweight='bold')
ax4.set_xlabel('Disease Status')
ax4.set_ylabel('Cholesterol (mg/dl)')
plt.setp(ax4.get_xticklabels(), rotation=0)

# Maximum heart rate by target
ax5 = fig.add_subplot(gs[1, 2])
df.boxplot(column='thalach', by='target', ax=ax5)
ax5.set_title('Max Heart Rate by Disease Status', fontweight='bold')
ax5.set_xlabel('Disease Status')
ax5.set_ylabel('Max Heart Rate')
plt.setp(ax5.get_xticklabels(), rotation=0)

# Feature correlation with target
ax6 = fig.add_subplot(gs[2, 0])
target_corr = corr_matrix['target'].drop('target').abs().sort_values(ascending=True)
target_corr.plot(kind='barh', ax=ax6, color='lightcoral')
ax6.set_title('Feature Correlation with Target', fontweight='bold')
ax6.set_xlabel('Absolute Correlation')

# Sex distribution by target
ax7 = fig.add_subplot(gs[2, 1])
sex_target = pd.crosstab(df['sex'], df['target'])
sex_target.plot(kind='bar', ax=ax7, color=['#ff9999', '#66b3ff'], alpha=0.7)
ax7.set_title('Disease Status by Sex', fontweight='bold')
ax7.set_xlabel('Sex (0=Female, 1=Male)')
ax7.set_ylabel('Count')
ax7.legend(['No Disease', 'Disease'])
ax7.set_xticklabels(['Female', 'Male'], rotation=0)

# Chest pain type by target
ax8 = fig.add_subplot(gs[2, 2])
cp_target = pd.crosstab(df['cp'], df['target'])
cp_target.plot(kind='bar', ax=ax8, color=['#ff9999', '#66b3ff'], alpha=0.7)
ax8.set_title('Disease Status by Chest Pain Type', fontweight='bold')
ax8.set_xlabel('Chest Pain Type')
ax8.set_ylabel('Count')
ax8.legend(['No Disease', 'Disease'])
ax8.set_xticklabels(['Typical Angina', 'Atypical Angina', 
                    'Non-anginal Pain', 'Asymptomatic'], rotation=45, ha='right')

plt.savefig('../Results/heart_disease_eda.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ EDA visualizations saved")


## 4. Data Preprocessing

We'll handle missing values, perform feature scaling, and split the data into training and testing sets. We'll also address class imbalance using SMOTE if necessary.


In [ ]:
# Handle missing values
df_processed = df.copy()
numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df_processed[col].isnull().sum() > 0:
        median_val = df_processed[col].median()
        df_processed[col].fillna(median_val, inplace=True)
        print(f"✓ Filled {col} missing values with median: {median_val:.2f}")

# Separate features and target
X = df_processed.drop('target', axis=1)
y = df_processed['target']

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining class distribution:\n{y_train.value_counts().to_dict()}")
print(f"Test class distribution:\n{y_test.value_counts().to_dict()}")

# Handle class imbalance using SMOTE (before scaling to ensure consistency)
imbalance_ratio = min(y_train.value_counts()) / max(y_train.value_counts())
print(f"\nClass imbalance ratio: {imbalance_ratio:.3f}")

if imbalance_ratio < 0.6:
    print("Applying SMOTE to balance training data...")
    smote = SMOTE(random_state=42)
    # Apply SMOTE to unscaled data first to ensure both scaled and unscaled versions match
    X_train, y_train = smote.fit_resample(X_train, y_train)
    print(f"✓ Training data after SMOTE: {X_train.shape[0]} samples")
    print(f"Balanced class distribution:\n{pd.Series(y_train).value_counts().to_dict()}")
else:
    print("Class imbalance is acceptable, skipping SMOTE")

# Feature scaling (after SMOTE to ensure scaled data matches resampled data)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("\n✓ Features scaled using StandardScaler")


## 9. Learning Curves

Learning curves help us understand if a model is overfitting or underfitting by showing how the model's performance changes as the training set size increases.

### Learning Curve for Best Model


In [ ]:
# NOTE: Learning curve code has been moved to Section 9 (after Model Evaluation)
# This ensures comparison_df and trained_models are available before execution


## 5. Model Training

We'll train five different machine learning models:
1. Decision Tree
2. Random Forest
3. Support Vector Machine (SVM)
4. Artificial Neural Network (ANN)
5. K-Nearest Neighbors (KNN)


In [ ]:
# Initialize models
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'SVM': SVC(random_state=42, probability=True, kernel='rbf'),
    'ANN': MLPClassifier(random_state=42, hidden_layer_sizes=(100, 50), 
                        max_iter=500, early_stopping=True),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# Store results
results = {}
trained_models = {}

# Train each model
for name, model in models.items():
    print(f"\n{'='*70}")
    print(f"Training {name}...")
    print(f"{'='*70}")
    
    # Select appropriate data (scaled for SVM, ANN, KNN)
    if name in ['SVM', 'ANN', 'KNN']:
        X_train_use = X_train_scaled
        X_test_use = X_test_scaled
    else:
        X_train_use = X_train.values
        X_test_use = X_test.values
    
    # Train model
    model.fit(X_train_use, y_train)
    print(f"✓ {name} trained successfully")
    
    # Make predictions
    y_pred = model.predict(X_test_use)
    y_pred_proba = model.predict_proba(X_test_use)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    # AUC-ROC
    auc_roc = None
    if y_pred_proba is not None:
        try:
            auc_roc = roc_auc_score(y_test, y_pred_proba)
        except:
            pass
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train_use, y_train, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    # Store results
    trained_models[name] = model
    results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'cv_mean': cv_mean,
        'cv_std': cv_std,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    if auc_roc:
        print(f"  AUC-ROC: {auc_roc:.4f}")
    print(f"  CV Score (mean ± std): {cv_mean:.4f} ± {cv_std:.4f}")

print("\n✓ All models trained successfully!")


## 11. Conclusion and Recommendations

Based on the comprehensive evaluation, we can now make data-driven recommendations for the best model for heart disease prediction in this context.

### Key Findings:
1. All models were successfully trained and evaluated
2. Hyperparameter tuning improved performance for Random Forest and SVM
3. The best model achieved high accuracy and F1-score
4. Learning curves help identify overfitting/underfitting issues

### Model Selection Justification:
The best model selection should consider:
- **Accuracy**: Overall correctness of predictions
- **Precision**: Minimizing false positives (critical in medical diagnosis)
- **Recall**: Minimizing false negatives (missing actual disease cases)
- **F1-Score**: Balanced measure of precision and recall
- **AUC-ROC**: Overall discriminative ability
- **Interpretability**: Ability to explain predictions (important for medical applications)
- **Generalization**: Performance on unseen data (from learning curves)

### Recommendations:
Based on the results, the recommended model should balance accuracy with interpretability, as medical professionals need to understand and trust the diagnostic system. The learning curve analysis helps ensure the model generalizes well to new data.


## 6. Hyperparameter Tuning

We'll perform hyperparameter tuning for Random Forest and SVM using GridSearchCV to optimize their performance.


In [ ]:
# 1. Random Forest Tuning
print("Tuning Random Forest...")
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_base = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(rf_base, rf_param_grid, cv=5, scoring='accuracy', 
                      n_jobs=-1, verbose=1)
rf_grid.fit(X_train.values, y_train)

print(f"\nBest parameters: {rf_grid.best_params_}")
print(f"Best CV score: {rf_grid.best_score_:.4f}")

# Evaluate tuned model
rf_tuned = rf_grid.best_estimator_
rf_tuned_pred = rf_tuned.predict(X_test.values)
rf_tuned_acc = accuracy_score(y_test, rf_tuned_pred)
print(f"Test accuracy: {rf_tuned_acc:.4f}")

# Update model and results
trained_models['Random Forest'] = rf_tuned
rf_tuned_proba = rf_tuned.predict_proba(X_test.values)[:, 1]
results['Random Forest'] = {
    'accuracy': rf_tuned_acc,
    'precision': precision_score(y_test, rf_tuned_pred, average='weighted', zero_division=0),
    'recall': recall_score(y_test, rf_tuned_pred, average='weighted', zero_division=0),
    'f1_score': f1_score(y_test, rf_tuned_pred, average='weighted', zero_division=0),
    'auc_roc': roc_auc_score(y_test, rf_tuned_proba),
    'y_pred': rf_tuned_pred,
    'y_pred_proba': rf_tuned_proba
}


In [ ]:
# 2. SVM Tuning
print("\nTuning SVM...")
svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'kernel': ['rbf', 'poly']
}

svm_base = SVC(random_state=42, probability=True)
svm_grid = GridSearchCV(svm_base, svm_param_grid, cv=5, scoring='accuracy',
                       n_jobs=-1, verbose=1)
svm_grid.fit(X_train_scaled, y_train)

print(f"\nBest parameters: {svm_grid.best_params_}")
print(f"Best CV score: {svm_grid.best_score_:.4f}")

# Evaluate tuned model
svm_tuned = svm_grid.best_estimator_
svm_tuned_pred = svm_tuned.predict(X_test_scaled)
svm_tuned_acc = accuracy_score(y_test, svm_tuned_pred)
print(f"Test accuracy: {svm_tuned_acc:.4f}")

# Update model and results
trained_models['SVM'] = svm_tuned
svm_tuned_proba = svm_tuned.predict_proba(X_test_scaled)[:, 1]
results['SVM'] = {
    'accuracy': svm_tuned_acc,
    'precision': precision_score(y_test, svm_tuned_pred, average='weighted', zero_division=0),
    'recall': recall_score(y_test, svm_tuned_pred, average='weighted', zero_division=0),
    'f1_score': f1_score(y_test, svm_tuned_pred, average='weighted', zero_division=0),
    'auc_roc': roc_auc_score(y_test, svm_tuned_proba),
    'y_pred': svm_tuned_pred,
    'y_pred_proba': svm_tuned_proba
}

print("\n✓ Hyperparameter tuning completed!")


## 7. Model Evaluation and Comparison

Let's create a comprehensive comparison of all models and identify the best performing one.


## 9. Learning Curves

Learning curves help us understand if a model is overfitting or underfitting by showing how the model's performance changes as the training set size increases.

### Learning Curve for Best Model


In [ ]:
# Select best model for learning curve
best_model_name = comparison_df.iloc[0]['Model']
best_model = trained_models[best_model_name]

# Determine which data to use
if best_model_name in ['SVM', 'ANN', 'KNN']:
    X_use = X_train_scaled
else:
    X_use = X_train.values

print(f"Generating learning curve for {best_model_name}...")

# Calculate learning curve
train_sizes, train_scores, val_scores = learning_curve(
    best_model, X_use, y_train, cv=5, 
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='accuracy', n_jobs=-1
)

# Calculate mean and std
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training Score', linewidth=2)
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.plot(train_sizes, val_mean, 'o-', color='red', label='Validation Score', linewidth=2)
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')

plt.xlabel('Training Set Size', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy Score', fontsize=12, fontweight='bold')
plt.title(f'Learning Curve - {best_model_name}', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../Results/learning_curve.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Learning curve saved")

# Interpretation
print("\nLearning Curve Interpretation:")
if train_mean[-1] > val_mean[-1] + 0.1:
    print("  - Model shows signs of overfitting (training score much higher than validation)")
elif val_mean[-1] < 0.7:
    print("  - Model shows signs of underfitting (both scores are low)")
else:
    print("  - Model shows good generalization (training and validation scores are close)")


In [ ]:
# Create comparison DataFrame
comparison_data = []
for name, metrics in results.items():
    comparison_data.append({
        'Model': name,
        'Accuracy': metrics['accuracy'],
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'F1-Score': metrics['f1_score'],
        'AUC-ROC': metrics.get('auc_roc', np.nan)
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Accuracy', ascending=False)

print("Model Comparison Summary:")
print(comparison_df.to_string(index=False))

# Identify best model
best_model_name = comparison_df.iloc[0]['Model']
print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Accuracy: {comparison_df.iloc[0]['Accuracy']:.4f}")
print(f"   F1-Score: {comparison_df.iloc[0]['F1-Score']:.4f}")

# Save comparison to CSV
comparison_df.to_csv('../Results/model_comparison.csv', index=False)
print("\n✓ Comparison results saved to CSV")


## 8. Visualizations

Let's create comprehensive visualizations to compare model performance.


In [ ]:
# ROC Curves
plt.figure(figsize=(10, 8))

for name, metrics in results.items():
    if metrics.get('y_pred_proba') is not None:
        fpr, tpr, _ = roc_curve(y_test, metrics['y_pred_proba'])
        auc = metrics.get('auc_roc', 0)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {auc:.3f})", linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
plt.title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../Results/roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ ROC curves saved")


In [ ]:
# Confusion Matrices
n_models = len(trained_models)
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, (name, metrics) in enumerate(results.items()):
    cm = confusion_matrix(y_test, metrics['y_pred'])
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Plot normalized confusion matrix
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
               ax=axes[idx], cbar_kws={'shrink': 0.8})
    axes[idx].set_title(f'{name}\nAccuracy: {metrics["accuracy"]:.3f}', 
                      fontweight='bold')
    axes[idx].set_xlabel('Predicted')
    axes[idx].set_ylabel('Actual')
    axes[idx].set_xticklabels(['No Disease', 'Disease'])
    axes[idx].set_yticklabels(['No Disease', 'Disease'])

# Hide unused subplot
if n_models < len(axes):
    axes[-1].axis('off')

plt.suptitle('Confusion Matrices - All Models', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../Results/confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Confusion matrices saved")


In [ ]:
# Model Comparison Bar Charts
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = plt.cm.Set3(np.linspace(0, 1, len(comparison_df)))

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    bars = ax.barh(comparison_df['Model'], comparison_df[metric], 
                  color=colors, alpha=0.7, edgecolor='black')
    ax.set_xlabel(metric, fontsize=11, fontweight='bold')
    ax.set_title(f'Model Comparison - {metric}', fontsize=12, fontweight='bold')
    ax.set_xlim([0, 1])
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, comparison_df[metric])):
        ax.text(val + 0.01, i, f'{val:.3f}', va='center', fontweight='bold')

plt.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../Results/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Model comparison charts saved")


In [ ]:
# Feature Importance for Tree-based Models
tree_models = ['Decision Tree', 'Random Forest']
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, name in enumerate(tree_models):
    if name in trained_models:
        model = trained_models[name]
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            feature_names = X_train.columns
            
            # Sort by importance
            indices = np.argsort(importances)[::-1]
            
            ax = axes[idx]
            ax.barh(range(len(importances)), importances[indices], 
                   color='steelblue', alpha=0.7, edgecolor='black')
            ax.set_yticks(range(len(importances)))
            ax.set_yticklabels([feature_names[i] for i in indices])
            ax.set_xlabel('Importance', fontsize=11, fontweight='bold')
            ax.set_title(f'{name} - Feature Importance', fontsize=12, fontweight='bold')
            ax.invert_yaxis()
            ax.grid(axis='x', alpha=0.3)

plt.suptitle('Feature Importance Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../Results/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Feature importance plots saved")


## 9. Conclusion and Recommendations

Based on the comprehensive evaluation, we can now make data-driven recommendations for the best model for heart disease prediction in this context.

### Key Findings:
1. All models were successfully trained and evaluated
2. Hyperparameter tuning improved performance for Random Forest and SVM
3. The best model achieved high accuracy and F1-score

### Model Selection Justification:
The best model selection should consider:
- **Accuracy**: Overall correctness of predictions
- **Precision**: Minimizing false positives (critical in medical diagnosis)
- **Recall**: Minimizing false negatives (missing actual disease cases)
- **F1-Score**: Balanced measure of precision and recall
- **AUC-ROC**: Overall discriminative ability
- **Interpretability**: Ability to explain predictions (important for medical applications)

### Recommendations:
Based on the results, the recommended model should balance accuracy with interpretability, as medical professionals need to understand and trust the diagnostic system.


## 10. Radar Chart Comparison

A radar chart provides a comprehensive view of model performance across multiple metrics simultaneously.


In [ ]:
# Create radar/spider chart for multi-metric comparison
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
num_metrics = len(metrics)

# Calculate angles for each metric
angles = np.linspace(0, 2 * np.pi, num_metrics, endpoint=False).tolist()
angles += angles[:1]  # Complete the circle

# Create figure
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

# Plot each model
colors = plt.cm.Set3(np.linspace(0, 1, len(comparison_df)))

for idx, row in comparison_df.iterrows():
    values = [row[metric] for metric in metrics]
    values += values[:1]  # Complete the circle
    
    ax.plot(angles, values, 'o-', linewidth=2, label=row['Model'], color=colors[idx])
    ax.fill(angles, values, alpha=0.15, color=colors[idx])

# Customize
ax.set_xticks(angles[:-1])
ax.set_xticklabels(metrics, fontsize=11, fontweight='bold')
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=9)
ax.grid(True, alpha=0.3)

plt.title('Model Performance Radar Chart', fontsize=16, fontweight='bold', pad=20)
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=9)
plt.tight_layout()
plt.savefig('../Results/radar_chart.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Radar chart saved")
